In [20]:
import pyspark
from delta import *
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import *

In [21]:
builder = pyspark.sql.SparkSession.builder.appName("DeltaTutorial") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

In [22]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [23]:
print("Starting Delta table creation")

data=[(1,"Anurag","Karki","Dhumbarahi",24),
      (2,"Anuska","Karki","Pragatitole",21),
      (3,"Ngawang","Gurung","Chandol",22),
      (4,"Beses","Kafle","Gatthaghar",23),
      (5,"Kalyan","Adhikari","Chabhil",26),
      (6,"Bishal","Neupane","Kapan",20)]

Starting Delta table creation


In [24]:
schema = StructType([
    StructField("Id", IntegerType(), True),
    StructField("FirstName", StringType(), True),
    StructField("LastName", StringType(), True),
    StructField("Location", StringType(), True),
    StructField("Age", IntegerType(), True)
])

In [25]:
sample_dataframe = spark.createDataFrame(data=data, schema=schema)

In [26]:
sample_dataframe.write.mode(saveMode="overwrite").format("delta").save("data/delta-table")

In [27]:
sample_dataframe.show()

+---+---------+--------+-----------+---+
| Id|FirstName|LastName|   Location|Age|
+---+---------+--------+-----------+---+
|  1|   Anurag|   Karki| Dhumbarahi| 24|
|  2|   Anuska|   Karki|Pragatitole| 21|
|  3|  Ngawang|  Gurung|    Chandol| 22|
|  4|    Beses|   Kafle| Gatthaghar| 23|
|  5|   Kalyan|Adhikari|    Chabhil| 26|
|  6|   Bishal| Neupane|      Kapan| 20|
+---+---------+--------+-----------+---+



In [14]:
sample_dataframe.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: integer (nullable = true)



In [28]:
deltaTable = DeltaTable.forPath(spark, "data/delta-table")

In [31]:
deltaTable.update(
    condition = expr('firstName == "Kalyan"'),
    set = {"id":lit(5),"firstName": lit("Kalyan"),"lastname":lit("Adhikaris"),"location":lit("Chabhils"),"age":lit(28)}
)

In [32]:
deltaTable.toDF().show()

+---+---------+---------+-----------+---+
| Id|FirstName| LastName|   Location|Age|
+---+---------+---------+-----------+---+
|  5|   Kalyan|Adhikaris|   Chabhils| 28|
|  2|   Anuska|    Karki|Pragatitole| 21|
|  1|   Anurag|    Karki| Dhumbarahi| 24|
|  4|    Beses|    Kafle| Gatthaghar| 23|
|  3|  Ngawang|   Gurung|    Chandol| 22|
|  6|   Bishal|  Neupane|      Kapan| 20|
+---+---------+---------+-----------+---+

